In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hys
import tkinter.filedialog as tkf
import ipywidgets as pyw
from sklearn.cluster import DBSCAN

In [ ]:
%matplotlib inline

In [ ]:
num_split = 3
adr = []
num_adr = []
for i in range(num_split):
    temp = tkf.askopenfilenames()
    num_adr.append(len(temp))
    adr.append(temp)
print(num_adr)
num_img = np.max(num_adr)

In [ ]:
data_by_split = []
data_flat = []
data_flat_by_split = []
data_shape = []
for i in range(num_split):
    temp = []
    temp_shape = []
    temp_flat = []
    for path in adr[i]:
        tmp = hys.load(path).data
        temp.append(tmp)
        temp_shape.append(tmp.shape)
        temp_flat.extend(tmp.reshape(-1, tmp.shape[2]).tolist())
        data_flat.extend(tmp.reshape(-1, tmp.shape[2]).tolist())
    data_by_split.append(temp)
    data_shape.append(temp_shape)
    data_flat_by_split.append(temp_flat)

In [ ]:
data_flat = np.asarray(data_flat)
print(data_flat.shape)
data_shape = np.asarray(data_shape)
print(data_shape.shape)
print(np.unique(data_shape[:, :, 2]))
s_dim = np.min(data_shape[:, :, 2])
print(s_dim)

In [ ]:
radial_var_sum = []
for arr in data_by_split:
    temp = []
    for img in arr:
        temp.append(np.mean(img, axis=(0, 1)))
    radial_var_sum.append(temp)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
linstyles = ["k-", "r-", "b-", "g-", "p-"]
for i in range(num_split):
    for line in radial_var_sum[i]:
        ax.plot(line, linstyles[i], label="split %d"%(i+1))
ax.grid()
fig.legend()
fig.tight_layout()
plt.show()

In [ ]:
radial_var_split = []
for i in range(num_split):
    radial_var_split.append(np.mean(radial_var_sum[i], axis=0))
    
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
for i in range(num_split):
    ax.plot(radial_var_split[i], linstyles[i], label="split %d"%(i+1))
ax.grid()
fig.legend()
fig.tight_layout()
plt.show()

In [ ]:
%matplotlib qt
binsize = 500
colors = ["black", "red", "blue", "green", "purple"]
fig1, ax1 = plt.subplots(1, 2, figsize=(15, 6))
fig2, ax2 = plt.subplots(num_split, num_img, figsize=(15, 15))
selected_img = []

def var_selector(k_range, var_range):
    r1 = k_range[0]
    r2 = k_range[1]
    v1 = var_range[0]
    v2 = var_range[1]
    ax1[0].cla()
    ax1[1].cla()
    selected_img.clear()
    for i in range(num_split):
        ax1[0].plot(radial_var_split[i], linstyles[i], label="split %d"%(i+1))
        ax1[1].hist(np.array(data_flat_by_split[i])[:, r1:(r2+1)].flatten(), bins=binsize, 
                    color=colors[i], alpha=0.6-0.15*i, label="split %d"%(i+1), density=True)
        for j in range(num_img):
            img = np.array(data_by_split[i][j])[:, :, r1:(r2+1)]
            img[np.where(img < v1)] = 0
            img[np.where(img > v2)] = 0
            img[np.where(img != 0)] = 1
            bool_img = np.sum(img, axis=2)
            bool_img = np.divide(bool_img, np.where(bool_img!=0, bool_img, 1))
            selected_img.append(bool_img)
            ind = num_img*i+j
            #print(ind)
            ax2.flat[ind].imshow(bool_img, cmap="afmhot")
            ax2.flat[ind].axis("off")
            ax2.flat[ind].set_title("%.2f %%"%(np.sum(bool_img)*100/(img.shape[0]*img.shape[1])))
                
    ax1[0].fill_between([r1, r2], np.max(radial_var_split), alpha=0.5, color="orange")
    ax1[0].grid()
    ax1[0].legend()
    ax1[1].fill_between([v1, v2], ax1[1].get_ylim()[1], alpha=0.5, color="orange")
    ax1[1].grid()
    ax1[1].legend()
    
    fig1.canvas.draw()
    fig2.canvas.draw()
    fig1.tight_layout()
    fig2.tight_layout()
    
range_widget = pyw.IntRangeSlider(value=[26, 29], min=0, max=s_dim-1, step=1)
range_widget_2 = pyw.FloatRangeSlider(value=[np.max(data_flat)*0.3, np.max(data_flat)*0.5], 
                                      min=np.min(data_flat), max=np.max(data_flat), step=0.01)
pyw.interact(var_selector, k_range=range_widget, var_range=range_widget_2)
plt.show()

In [ ]:
k_range = range_widget.value
print(k_range)
v_range = range_widget_2.value
print(v_range)

def cluster_size(img):
    label = np.unique(img)
    label =  label[label != -1]
    print(label)
    cluster_size = []
    for i in label:
        cluster_size.append(len(np.where(img==i)[0]))
    cluster_size = np.sort(cluster_size)
    print(cluster_size)

    largest = np.around(np.sqrt(cluster_size[-1]), decimals=2) 
    smallest = np.around(np.sqrt(cluster_size[0]), decimals=2)
    avg = np.around(np.sqrt(np.mean(cluster_size)), decimals=2)
    
    return avg, largest, smallest

In [ ]:
%matplotlib qt
fig3, ax3 = plt.subplots(1, 3, figsize=(10, 10))
row_label = ["area (%)", "# of clusters", "mean size (pixels)", "largest size (pixels)", "smallest size (pixels)"]

def selector(e, split, n_img):
    db = DBSCAN(eps=e, min_samples=4)
    ind = num_img*(split-1)+n_img-1

    ax3[0].cla()
    ax3[1].cla()
    ax3[2].cla()
    
    ax3[0].imshow(selected_img[ind], cmap="afmhot")
    ax3[0].axis("off")  
    
    sel_coor = np.nonzero(selected_img[ind])
    X = np.stack((sel_coor[0], sel_coor[1]), axis=1)
    db.fit(X)
    label = db.labels_
    img_tmp = np.zeros_like(selected_img[ind])
    img_tmp += -1
    img_tmp[X[:, 0], X[:, 1]] = label
    bool_img = np.where(img_tmp<0, img_tmp, 1)
    bool_img = np.where(bool_img>-1, bool_img, 0)
    ax3[1].set_title("split %d, image %d, epsilon=%.1f"%(split, n_img, e), fontsize=15)
    ax3[1].imshow(bool_img, cmap="afmhot")
    ax3[1].axis("off")
    
    area = np.around(100 * np.sum(bool_img) / (img_tmp.shape[0]*img_tmp.shape[1]), decimals=2)
    n_cluster = len(np.unique(label)) - 1
    avg, largest, smallest = cluster_size(img_tmp)
    table_vals = [[area], [n_cluster], [avg], [largest], [smallest]]

    table = ax3[2].table(cellText=table_vals, rowLabels=row_label, loc="center", rowLoc="center", cellLoc="center", colWidths=[0.1]*3)
    table.scale(3, 8)
    table.auto_set_font_size(False)
    table.set_fontsize(25)
    ax3[2].axis("off")
    fig3.tight_layout()
    
eps_widget = pyw.FloatSlider(value=1.5, min=1.0, max=2.0, step=0.1, description="epsilon: ")
split_widget = pyw.Dropdown(options=np.arange(num_split)+1, value = 1, description="split: ")
n_img_widget = pyw.Dropdown(options=np.arange(np.max(num_adr))+1, value = 1, description="image: ")

pyw.interact(selector, e=eps_widget, split=split_widget, n_img=n_img_widget)
fig3.show()